In [40]:
import ast
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import KFold
import joblib

In [ ]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

"""
pra esse modelo eu testei 2 formas do target.
a primeira, sendo uma lista de 30 valores, com cada um deles representando a acurácia do ansatz.
e o target de cada ansatz é definido pelo seu valor dividido pela soma da acurácia de todos os ansatz
a segunda forma do target, é uma lista de 30 valores, preenchidos com 0.0, exceto no ansatz que teve a melhor 
acurácia.
ambas geraram outputs distintos para o mesmo caso, que seria a entrada de index 6 dos dados, e tiveram scores
diferentes em suas validações cruzadas.
a primeira forma teve score aproximado de 0.8, e apresentou os ansatz 14, 11, 20 como melhores,
não posso afirmar que são probabilidades o output desse modelo já que provavelmente essa soma de valores ultrapassa 1
na segunda forma teve score muito baixo de 0.08, e apresentou os ansatz 0, 2, 11 como melhores.
a segunda forma apresentou resultados muito aproximados do classifier olhando os melhores ansatz, que foi 0, 2, 8
irei validar posteriormente os resultados e farei testes para os 2 casos de inputs e ver qual foi a melhor forma no final
por enquanto apenas a primeira será utilizada
"""
#primeira
y_prob = y / y.sum(axis=1, keepdims=True)

#segunda
y_best_ansatz = np.argmax(y, axis=1)  # Retorna índices 0-29

y = np.zeros(shape=(360, 30), dtype=np.float64)

for i in range(360):
    y[i][y_best_ansatz[i]] = 1.0

In [42]:
# criando kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Definir os parâmetros a testar
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10, 15, 20, 25, 30],
    'min_samples_leaf': [1, 2, 4, 5,  10, 15, 20, 30, 35, 40],
    'criterion': ['squared_error', 'friedman_mse']
}

dt_regressor_classifier = DecisionTreeRegressor(random_state=45)

grid_search = GridSearchCV(dt_regressor_classifier, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=12)

grid_search.fit(X, y_prob)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)

Melhores parâmetros: {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2}


In [43]:
# Usar os melhores parâmetros encontrados para validação cruzada
best_dt_regressor_classifier = grid_search.best_estimator_

score = cross_val_score(best_dt_regressor_classifier, X, y_prob, cv=kf)
print("Scores:", score)
print("Média:", score.mean())
print("Desvio padrão:", score.std())


Scores: [0.80291593 0.80676946 0.77695103]
Média: 0.7955454737531786
Desvio padrão: 0.013242037984241688


In [44]:
# treinando o dt classifier e obtendo os top3 ansatz mais classificados
best_dt_regressor_classifier.fit(X, y_prob)


# 3. Prever e rankear
def predict_rank_ansatz(model, X_new, k=3):
    probas = model.predict(X_new.reshape(1, -1))[0]
    topk = np.argsort(probas)[-k:][::-1]
    return topk, probas[topk]

# Rankear os top-3 ansatz com maiores probabilidades
top3_idx, top3_probs = predict_rank_ansatz(best_dt_regressor_classifier, X[6])

print(f"Top-3 Ansatz: {top3_idx} com probabilidades {top3_probs}")

Top-3 Ansatz: [14 11 20] com probabilidades [0.03719537 0.03694778 0.03689234]


In [45]:
# salvar o modelo treinado
joblib.dump(best_dt_regressor_classifier, './models_salvos/dt_regressor_classifier.joblib')

['./models_salvos/dt_regressor_classifier.joblib']

Exception ignored in: <function ResourceTracker.__del__ at 0x7395eae92de0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x761176e7ade0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7083da486de0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/reso